In this notebook, we'll actually implement an ML model and see how sklearn handles making predictions on data. 

In [7]:
import pandas as pd
import sklearn

Read in our data file:

In [8]:
df = pd.read_csv('https://raw.githubusercontent.com/benartuso/launch-data-science/main/week-3/breast-cancer.csv')

In [9]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


### Preprocessing
Let's repeat the steps we did yesterday.

In [10]:
#Isolate our X variables
X = df.drop(columns=['id', 'diagnosis'])

In [11]:
#Isolate y variable:
y = df.diagnosis

In [12]:
#What's going on with the y variable?
df.diagnosis.value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

Benign = not cancerous, malignant = cancerous.

We can only have one output column, so instead of one-hot encoding, we'll use the map function. 

In [15]:
#map..
our_map = {'M':1, 'B':0}
y = y.map(our_map)

### Preprocessing

In [23]:
#Split our data!
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=811, test_size=0.2)

In [30]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()

X_train = imputer.fit_transform(X_train)
X_train = scaler.fit_transform(X_train)

X_val = imputer.transform(X_val)
X_val = scaler.transform(X_val)

### Modeling using a basic decision tree

In [31]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()


Fit our model on our training x data and the associated labels...

In [32]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [33]:
#Now, we can predict on the training set...
training_preds = tree.predict(X_train)
val_preds = tree.predict(X_val)

In [36]:
#And compare them to the labels...
from sklearn.metrics import accuracy_score
print("Train accuracy:")
accuracy_score(training_preds, y_train)

Train accuracy:


1.0

In [37]:
print("Val accuracy:")
accuracy_score(val_preds, y_val)

Val accuracy:


0.8947368421052632

In [50]:
X = X.drop(columns='Unnamed: 32')

In [65]:
#WE seem to be waaaay overfitting here. Let's see if Logistic regression will work better!
from sklearn.ensemble import RandomForestClassifier

In [66]:
from sklearn.linear_model import LogisticRegression
rf = LogisticRegression()
rf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [67]:
print(accuracy_score(rf.predict(X_train), y_train))
print(accuracy_score(rf.predict(X_val), y_val))

0.9912087912087912
0.9649122807017544
